# 📍 Pose Estimation - 2D Keypoint Extraction Using YOLOv8

This notebook performs 2D human pose estimation on exercise images using the YOLOv8 pose model. It detects joints such as shoulders, elbows, hips, knees, and ankles, and extracts their coordinates for further biomechanical analysis.

The extracted keypoints are saved in a structured CSV file for use in subsequent steps like joint angle computation and load estimation.

**Components**:
- Dataset upload (manual image or ZIP)
- YOLOv8 model loading and inference
- Keypoint extraction (COCO format: 17 landmarks)
- Annotated image saving
- Output CSV with joint coordinates

# 1. Installing Required Packages

In [ ]:
# Install Ultralytics YOLO and supporting libraries
!pip install ultralytics
!pip install opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# 2. Importing Libraries

In [ ]:
# Import essential libraries
from ultralytics import YOLO
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
import zipfile
import pandas as pd
import math

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# 3. Load YOLOv8 Pose Model (Run This Once)

In [ ]:
# Load the YOLOv8 pose model (n = nano version, fast and light)
model = YOLO("yolov8l-pose.pt")

100%|██████████| 85.3M/85.3M [00:01<00:00, 84.7MB/s]


# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
output_dir = "/content/drive/MyDrive/SEE_Assessment/outputs"

In [ ]:
output_dir = "/content/drive/MyDrive/SEE_Assessment/outputs"
os.makedirs(output_dir, exist_ok=True)

# 4A. Option 1 for a Single or Few Images (Manual Upload)

In [ ]:
# Upload one or more images from your local machine
uploaded = files.upload()

# Automatically get the uploaded image path
image_files = list(uploaded.keys())

    # Loop through all uploaded images
for image_path in image_files:
    # Run YOLOv8 pose inference
    results = model(image_path)

    # Display annotated image (inline in Colab)
    #results[0].show()
    annotated = results[0].plot()

    # Save to Google Drive
    save_path = os.path.join(output_dir, f"annotated_{image_path}")
    cv2.imwrite(save_path, annotated)

    # Optional: Show in Colab
    from google.colab.patches import cv2_imshow
    cv2_imshow(annotated)

    print(f"✅ Saved: {save_path}")

# 4B. Option 2 for Dataset (Upload ZIP & Batch Process)

In [ ]:
uploaded = files.upload()  # Upload your ZIP file dataset

In [ ]:
zip_name = list(uploaded.keys())[0]

# Unzip dataset
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset/")

# Define dataset folder path
dataset_folder = "/content/dataset/PoseEstimation/PE/"
image_files = [f for f in os.listdir(dataset_folder) if f.endswith(('.jpg', '.png'))]

# Define outputs folder in current session for quick access
session_output = "/content/outputs"
os.makedirs(session_output, exist_ok=True)

# Loop through dataset images, run YOLO once, and save results to both destinations
for image_file in image_files:
    image_path = os.path.join(dataset_folder, image_file)
    results = model(image_path)

    # Annotated image from YOLO
    annotated = results[0].plot()

    # Save to Colab session (for current notebook viewing)
    session_save_path = os.path.join(session_output, f"annotated_{image_file}")
    cv2.imwrite(session_save_path, annotated)

    # Save to Google Drive (persistent)
    drive_save_path = os.path.join(output_dir, f"annotated_{image_file}")
    cv2.imwrite(drive_save_path, annotated)

    print(f"✅ Saved to session: {session_save_path}")
    print(f"✅ Saved to Drive:   {drive_save_path}")

    # Optional: Show in notebook
    cv2_imshow(annotated)

# 5. Extract and Organize 2D Keypoints from YOLOv8 pose results

In [ ]:
COCO_KEYPOINTS = {
    0: 'nose', 1: 'left_eye', 2: 'right_eye', 3: 'left_ear', 4: 'right_ear',
    5: 'left_shoulder', 6: 'right_shoulder', 7: 'left_elbow', 8: 'right_elbow',
    9: 'left_wrist', 10: 'right_wrist', 11: 'left_hip', 12: 'right_hip',
    13: 'left_knee', 14: 'right_knee', 15: 'left_ankle', 16: 'right_ankle'
}

Determine Input Source & Collect Image Paths

In [ ]:
# ✅ Paths to check
option1_dir = "/content"
dataset_folder = "/content/dataset/PoseEstimation/PE/"
drive_output_dir = "/content/drive/MyDrive/SEE_Assessment/outputs"

# ✅ Gather files from Option 1: Manually uploaded images
option1_images = {
    f: os.path.join(option1_dir, f)
    for f in os.listdir(option1_dir)
    if f.lower().endswith(('.jpg', '.png'))
}

# ✅ Gather files from Option 2: ZIP dataset images
option2_images = {}
if os.path.exists(dataset_folder):
    option2_images = {
        f: os.path.join(dataset_folder, f)
        for f in os.listdir(dataset_folder)
        if f.lower().endswith(('.jpg', '.png'))
    }

# ✅ Gather files from Drive output folder
drive_images = {}
if os.path.exists(drive_output_dir):
    drive_images = {
        f: os.path.join(drive_output_dir, f)
        for f in os.listdir(drive_output_dir)
        if f.lower().endswith(('.jpg', '.png'))
    }

# ✅ Merge all sources into a single dictionary (deduplicated by filename)
# Priority order: Drive > Dataset > Manual upload
merged_images = {}
merged_images.update(option1_images)       # Lowest priority
merged_images.update(option2_images)       # Overwrite duplicates from Option 1
merged_images.update(drive_images)         # Overwrite duplicates from above

# ✅ Final list of unique image paths
all_image_paths = list(merged_images.values())

# ✅ Report
print(f"📸 Total unique images to process: {len(all_image_paths)}")

In [ ]:
# Check for any duplicate filenames
all_filenames = list(merged_images.keys())
unique_filenames = set(all_filenames)

if len(all_filenames) != len(unique_filenames):
    print("⚠️ Warning: Duplicate filenames detected even after deduplication!")
else:
    print("✅ No duplicate filenames detected.")

In [ ]:
from collections import defaultdict

# Track which sources contain each filename
filename_sources = defaultdict(list)

for f in option1_images: filename_sources[f].append("manual")
for f in option2_images: filename_sources[f].append("dataset")
for f in drive_images:   filename_sources[f].append("drive")

# Print duplicates across sources
print("\n🔁 Files found in more than one location:")
duplicates = {f: srcs for f, srcs in filename_sources.items() if len(srcs) > 1}

if duplicates:
    for filename, sources in duplicates.items():
        print(f"📛 {filename} -> {sources}")
    print(f"\n❗ Total duplicated filenames: {len(duplicates)}")
else:
    print("✅ No filename overlaps across sources.")


Extract and Organize Keypoints for ALL Images (2 Options Combined)

In [ ]:
# Create a list to store keypoints for all images
keypoint_records = []

for image_path in all_image_paths:
    results = model(image_path)

    keypoints = results[0].keypoints.xy.cpu().numpy()

    if keypoints.shape[0] == 0:
        print(f"No person detected in {image_path}")
        continue

    # Use first person only
    person_kpts = keypoints[0]  # Shape: (17, 2)

    # Extract file name only (not full path)
    row = {'image': os.path.basename(image_path)}

    for i, (x, y) in enumerate(person_kpts):
        joint_name = COCO_KEYPOINTS[i]
        row[f"{joint_name}_x"] = float(x)
        row[f"{joint_name}_y"] = float(y)

    keypoint_records.append(row)

In [ ]:
# Create DataFrame
df_keypoints = pd.DataFrame(keypoint_records)

# Check for duplicate image entries
dupes = df_keypoints['image'].duplicated().sum()

if dupes > 0:
    print(f"⚠️ Warning: Found {dupes} duplicate filename(s) in keypoints.csv!")
    # Optional: drop duplicates before saving
    df_keypoints = df_keypoints.drop_duplicates(subset='image', keep='first')
else:
    print("✅ No duplicate image entries found.")

print("Keypoints DataFrame:")
df_keypoints.head()


Keypoints DataFrame:


,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,aaron-brogden-9y4MaTz2Js0-unsplash.jpg,1401.332642,1597.854248,1446.944336,1525.367065,1358.923218,1488.737671,0.00000,0.000000,1135.094727,...,1019.826477,3109.603271,1205.262939,3840.495361,1034.875000,3934.542969,0.000000,0.000000,0.000000,0.000000
1,john-fornander-TAZoUmDqzXk-unsplash.jpg,2121.877930,3358.743652,2171.137939,3230.602295,2080.133301,3278.068848,2452.16333,3098.942627,0.000000,...,2376.989746,5200.759277,2894.450195,6393.967285,2422.261963,6286.392090,3463.841553,7189.644043,2639.279541,7349.532715
2,pexels-andrewperformance1-733500.jpg,821.343445,650.343262,915.683655,526.811462,710.481079,539.409058,1056.92688,526.449890,571.784607,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,pexels-johnny-garcia-1041229-2011377.jpg,1792.022217,974.793030,0.000000,0.000000,1687.227295,848.847595,0.00000,0.000000,1306.626221,...,780.840820,2661.416260,2092.994629,2757.151123,1876.455933,2851.715088,3121.374268,2897.208252,3012.409912,3216.100098
4,pexels-shvetsa-4587381.jpg,2427.232178,996.484314,2471.211182,882.327026,2359.252197,871.895264,0.00000,0.000000,2057.656738,...,1895.375488,2790.837646,3195.959717,2483.524170,1931.501587,3489.845703,2956.810303,3543.261475,0.000000,0.000000


In [ ]:
df_keypoints.to_csv("/content/keypoints.csv", index=False)
print("Saved keypoints to /content/keypoints.csv")

Saved keypoints to /content/keypoints.csv


In [ ]:
# Save final keypoints data
df_keypoints.to_csv("/content/drive/MyDrive/SEE_Assessment/keypoints.csv", index=False)
print("Saved keypoints.csv to Drive.")